In [1]:
from __future__ import annotations # To prevent Subscript for class "list" will generate runtime exception; enclose type annotation in quotes
import seaborn as sns
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import scipy
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from Kmeans import *
from decimal import Decimal
from SVD import * 
%matplotlib inline
# pd.options.display.max_rows = 4000
# pd.options.display.max_columns = 4000

In [2]:
df = pd.read_csv('preprocessed.csv',index_col=0)
Genres=list(pd.read_csv('genre_order.csv',index_col=0)['0'])
df

,Gender,Age,Zip-code_1,Zip-code_2,Zip-code_3,Zip-code_4,Zip-code_5,Zip-code_6,Zip-code_7,Zip-code_8,...,Action,1970s,Animation,Comedy,Thriller,Mystery,1930s,Children's,Film-Noir,Drama
UserID,,,,,,,,,,,,,,,,,,,,,
1,0,0.017857,0,0,0,1,0,0,0,0,...,0.182449,7.981933e-02,0.645186,0.504759,0.091603,0.073792,6.841887e-02,0.747438,0.004701,0.822336
2,1,1.000000,0,0,0,0,0,0,1,0,...,0.607944,8.810512e-02,0.027614,0.273131,0.330404,0.022027,8.810177e-03,0.057179,0.004405,1.000000
3,1,0.446429,0,0,0,0,1,0,0,0,...,0.814809,1.975349e-01,0.086428,1.000000,0.148152,0.000007,7.633480e-02,0.086428,0.125999,0.271605
4,1,0.803571,0,1,0,0,0,0,0,0,...,0.351200,9.242126e-02,0.190780,1.000000,0.043130,0.166454,1.559110e-07,0.000000,0.100681,0.098582
5,1,0.446429,0,0,0,0,1,0,0,0,...,0.125002,8.925164e-07,0.026041,0.346351,0.182294,0.039064,2.604452e-03,0.039062,0.018229,0.562499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0,0.446429,0,0,1,0,0,0,0,0,...,0.294317,1.626059e-01,0.091950,0.552734,0.301040,0.075498,5.321991e-02,0.123925,0.046457,0.898209
6037,0,0.803571,0,0,0,0,0,0,1,0,...,0.257441,3.536907e-01,0.003553,0.535784,0.650177,0.117847,3.926586e-02,0.049999,0.071433,1.000000
6038,0,1.000000,1,0,0,0,0,0,0,0,...,0.062499,2.187714e-01,0.187502,1.000000,0.794578,0.255207,0.000000e+00,0.000000,0.134377,0.750100


In [3]:
num_cluster=26
to_remove=10
threshold=0

In [4]:
from SVD import * 
u,s,sigma,V_trans = ReducedSVD(df.to_numpy(),threshold,num_cluster)

In [5]:
df = pd.DataFrame(u@sigma@V_trans,columns=df.columns)

In [6]:
def cluster_to_matrix(X_train,y_train,num_cluster):
    k=Kmeans(K=num_cluster,iter=150,p=2,random_state=42)
    y_pred,_=k.predict(np.array(X_train),choice=0)
    k.centroids=np.array(k.centroids)
    empty_cluster = []
    for idx,i in enumerate(k.clusters):
        if len(i)==0:
            empty_cluster.append(idx)
    X_train2 = pd.concat([X_train,y_train,pd.DataFrame(y_pred,columns=["label"],dtype='int')],axis="columns")
    df_cluster_rep = X_train2.groupby(['label'], as_index=False).mean()
    df_cluster_rep.drop(labels="label",axis="columns",inplace=True)
    matrix = df_cluster_rep.to_numpy()
    final_matrix = []
    j = 0 
    for i in range(num_cluster):
        if i in empty_cluster:
            final_matrix.append(np.zeros(len(df.columns)))
        else:
            final_matrix.append(matrix[j])
            j+=1

    return np.array(final_matrix),k 

In [7]:
x_col_size=len(df.columns)-len(Genres)+6

In [8]:
x_col_size

38

In [9]:
y = df[df.columns[x_col_size:]]
x = df.drop(labels=df.columns[x_col_size:],axis="columns")

In [10]:
x

,Gender,Age,Zip-code_1,Zip-code_2,Zip-code_3,Zip-code_4,Zip-code_5,Zip-code_6,Zip-code_7,Zip-code_8,...,Occupation_17,Occupation_18,Occupation_19,Occupation_20,Documentary,Sci-Fi,Musical,1950s,War,1980s
0,0.069829,-0.072083,-0.113767,-0.122070,0.087525,1.004756,-0.035548,-0.008877,0.154174,0.050547,...,0.115621,-0.002879,0.046349,-0.021184,0.186001,0.101797,0.289287,0.193447,0.278310,0.409302
1,1.092761,0.989783,-0.160943,-0.169362,0.211664,0.048528,-0.022921,0.039462,1.317553,0.148210,...,0.257307,0.012978,0.047589,0.025140,0.230631,0.128549,-0.159575,0.251703,0.417598,0.550903
2,1.079500,0.540595,-0.149427,-0.128898,0.142298,0.004297,0.972808,0.027298,0.253278,0.103164,...,0.214026,0.055874,0.057508,0.044148,0.205946,0.100651,-0.049046,0.214743,0.378320,0.924191
3,1.014257,0.750062,-0.033623,0.962153,-0.020921,-0.023640,-0.026979,-0.024008,-0.010927,-0.017876,...,-0.012363,0.008182,-0.009726,-0.033459,0.087820,0.130279,0.087748,0.159754,0.060616,0.266299
4,1.044418,0.466520,-0.091616,-0.092863,0.070799,-0.000164,0.968654,-0.004924,0.119960,0.043711,...,0.102567,0.027890,0.035503,0.998106,0.116890,0.050038,-0.048100,0.095434,0.136126,0.132567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.055732,0.541346,-0.126841,-0.111835,1.095681,-0.012113,-0.036362,0.016278,0.184604,0.075720,...,0.172255,0.018742,0.054181,0.028958,0.128877,0.135838,-0.057896,0.150202,0.292573,0.574681
6036,0.044226,0.808357,-0.095478,-0.098573,0.093853,0.010908,-0.026662,0.010401,1.142366,0.059316,...,0.120655,-0.004706,0.016732,0.010564,0.165301,0.354072,-0.013671,0.235313,0.277828,0.512587
6037,0.026167,0.946492,0.976563,-0.024143,0.071578,0.033990,0.021586,0.026410,0.094949,0.055185,...,0.081697,-0.005149,0.004777,0.019251,0.188080,0.053697,0.201107,0.310792,0.176017,0.492293
6038,-0.006279,0.801542,0.985330,-0.010833,0.005158,-0.000543,-0.000468,0.003222,0.009283,0.005434,...,0.005149,-0.013875,-0.012669,0.002008,0.138490,0.129023,0.294097,0.312165,0.166750,0.270027


In [11]:
y

,Western,Horror,1990s,2000s,1940s,Crime,Romance,Fantasy,1920s,Adventure,...,Action,1970s,Animation,Comedy,Thriller,Mystery,1930s,Children's,Film-Noir,Drama
0,0.088240,0.197183,1.231563,0.263841,0.153906,0.203854,0.355653,0.264059,0.035597,0.557392,...,0.287701,0.054779,0.569476,0.480615,0.025060,0.124030,0.124582,0.786609,0.172444,0.979891
1,0.093148,-0.070684,1.358922,0.112905,0.193882,0.249127,0.406260,0.012241,0.050707,0.723463,...,0.817770,0.124057,0.038066,0.118098,0.208154,0.137678,0.150504,0.009842,0.255555,1.193061
2,0.123845,0.049840,1.055981,0.156259,0.185123,0.173449,0.189311,0.123254,0.033328,0.914627,...,1.221109,0.361742,0.127782,0.887535,0.090600,0.102639,0.139996,0.187103,0.213382,0.451126
3,0.084816,-0.091116,0.225070,0.169460,0.190667,0.084869,0.098000,0.058117,0.069068,0.154541,...,0.219463,0.168958,0.054362,0.891731,0.062091,0.118131,0.116654,0.070101,0.133533,0.018217
4,0.045969,0.021686,1.185075,0.069625,0.072665,0.144955,0.210633,0.015860,0.021824,0.272070,...,0.221061,-0.002357,0.037221,0.298964,0.168183,0.044982,0.062933,0.026125,0.108624,0.664269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.036617,0.213663,1.251022,0.075525,0.089998,0.167931,0.242163,0.087260,0.005862,0.530229,...,0.536357,0.302205,0.100985,0.480078,0.188187,0.071263,0.082490,0.148567,0.137043,1.057633
6036,0.081490,0.088932,0.746048,0.177220,0.190611,0.340484,0.295870,0.023898,0.041893,0.325030,...,0.403229,0.398822,0.020832,0.449244,0.455725,0.205122,0.140224,-0.030050,0.226519,1.096550
6037,0.133169,0.055805,0.650666,0.364059,0.333251,0.480560,0.570248,0.052708,0.077317,0.064259,...,0.223750,0.196022,0.142564,0.962642,0.670287,0.351427,0.203689,0.145357,0.324538,0.839916
6038,0.131164,-0.022055,0.365860,0.233191,0.283037,0.173815,0.539753,0.123247,0.064836,0.185125,...,0.108611,0.320670,0.213264,0.941236,0.126262,0.199058,0.170061,0.290361,0.193101,0.429405


In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.05, shuffle=True, random_state=42)

In [13]:
matrix,kmeans_model = cluster_to_matrix(x_train,y_train,num_cluster)

In [14]:
matrix.shape

(26, 60)

In [15]:
def predict(datapoint,kmeans,matrix_clean):
    y_pred = kmeans.predictPoint(datapoint)
    inferred_full = matrix_clean[y_pred]
    inferred = inferred_full[x_col_size:]
    inferred_full = inferred_full[len(df.columns)-len(Genres):]
    sorted_ratings = []
    sorted_ratings_full = []
    for idx,element in enumerate(inferred):
        sorted_ratings.append([element,idx+x_col_size])
    for idx,element in enumerate(inferred_full):
        sorted_ratings_full.append([element,idx+len(df.columns)-len(Genres)])
    sorted_ratings.sort()
    sorted_ratings.reverse()
    sorted_ratings_full.sort()
    sorted_ratings_full.reverse()
    return sorted_ratings, sorted_ratings_full

In [16]:
x_test.to_numpy()[0][:x_col_size]
sorted_ratings, sorted_ratings_full = predict(x_test.to_numpy()[0][:x_col_size],kmeans_model,matrix)
s="Top 5 PREDICTED recommended genres are:\n"
for i in range(5):
    s+=df.columns[sorted_ratings_full[i][1]]+"\n"
print(s)

Top 5 PREDICTED recommended genres are:
1990s
Drama
Action
1980s
Comedy



In [17]:
sorted_ratings_actual=[]
inferred = y_test.to_numpy()[0]
for idx,element in enumerate(inferred):
    sorted_ratings_actual.append([element,idx+x_col_size])
sorted_ratings_actual.sort()
sorted_ratings_actual.reverse()
s="Top 5 ACTUAL recommended genres are:\n"
for i in range(5):
    s+=df.columns[sorted_ratings_actual[i][1]]+"\n"
print(s)

Top 5 ACTUAL recommended genres are:
1990s
Drama
Comedy
Romance
Adventure



In [18]:
rating_dict={}
for i in range(len(sorted_ratings_full)):
    rating_dict[df.columns[sorted_ratings_full[i][1]]]=sorted_ratings_full[i][0]
rating_dict

{'1990s': 0.9562595253409042,
 'Drama': 0.7326015420194506,
 'Action': 0.541950856482205,
 '1980s': 0.47918326770482167,
 'Comedy': 0.4697051555799542,
 'Adventure': 0.46572033981335537,
 'Romance': 0.303346271119026,
 'Thriller': 0.25810933950073905,
 'War': 0.2553879106753539,
 'Sci-Fi': 0.24293913688540714,
 'Crime': 0.22707820414530078,
 '1970s': 0.214318060644805,
 '1950s': 0.21430181129898299,
 'Film-Noir': 0.19452367308833746,
 '2000s': 0.19341343508620581,
 '1940s': 0.18500934075871056,
 'Documentary': 0.1758239906508216,
 'Mystery': 0.14875456931875483,
 '1930s': 0.13533954138559381,
 "Children's": 0.13440441492564598,
 'Animation': 0.12932287442592957,
 'Horror': 0.12490743228188869,
 'Fantasy': 0.10616152261801512,
 'Western': 0.09920309527938205,
 '1920s': 0.049577730737241484,
 'Musical': 0.029440221999601897,
 '1960s': 0.024510611541953657,
 '1910s': -0.00783615364575778}

In [19]:
def predict_for_dataset(X_test,kmeans_instance,matrix_clean):
    X_test = X_test.to_numpy()
    y_pred = []
    for i in range(X_test.shape[0]):
        datapoint = X_test[i]
        y_pred.append(predict(datapoint,kmeans_instance,matrix_clean)[0])
    return np.array(y_pred)

In [20]:
x_test

,Gender,Age,Zip-code_1,Zip-code_2,Zip-code_3,Zip-code_4,Zip-code_5,Zip-code_6,Zip-code_7,Zip-code_8,...,Occupation_17,Occupation_18,Occupation_19,Occupation_20,Documentary,Sci-Fi,Musical,1950s,War,1980s
5529,0.039950,0.332992,-0.069652,-0.069368,0.061342,0.005114,-0.019324,0.999255,0.100688,0.036731,...,0.082612,-0.007015,0.015369,0.003575,0.092381,0.072025,0.020354,0.101334,0.147379,0.440737
710,1.042833,0.538179,-0.107154,-0.100232,0.108460,0.008441,-0.010753,0.022681,0.178242,1.077048,...,0.190393,0.024105,0.043082,0.077667,0.146353,0.174904,-0.061431,0.207793,0.297112,0.709458
4923,0.037909,0.592096,-0.060634,-0.064646,0.034419,-0.006196,-0.024412,-0.010681,0.068002,0.020450,...,0.057941,-0.002007,0.008844,-0.011061,0.145638,0.020958,0.043516,0.189957,0.188127,0.381257
2153,1.072727,0.459106,-0.119753,-0.123694,0.152024,0.031423,-0.022203,1.018737,0.218180,0.103367,...,0.184622,0.007006,0.045544,0.016035,0.252355,0.230949,0.028317,0.317439,0.327647,0.507509
1272,1.025694,0.651387,0.953951,-0.046599,0.074730,0.021211,-0.001107,0.021494,0.106570,0.049727,...,0.076661,0.015725,0.033577,0.006772,0.120820,0.090561,0.022291,0.197911,0.208787,0.496097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,0.015260,0.489585,-0.077160,-0.085001,0.041071,0.005574,-0.030741,-0.006283,0.062382,0.028083,...,0.103920,0.026098,0.033847,0.045856,0.171526,0.106363,-0.001463,0.196941,0.074684,0.183739
4648,1.041921,1.188213,0.932069,-0.061131,0.094831,0.016170,-0.008996,0.000532,0.108679,0.033664,...,0.074965,0.045039,0.015494,-0.005647,0.117221,0.053135,0.065959,0.246861,0.337969,0.255392
323,1.037320,0.681487,-0.107677,-0.110583,0.151211,0.036825,0.992737,0.039384,0.219849,0.108581,...,1.181305,0.020245,0.030033,0.035487,0.251451,0.208112,0.117006,0.264187,0.286901,0.292324
2943,0.030340,0.336841,-0.044757,-0.056951,0.068674,1.013809,-0.006882,0.018093,0.104288,0.055793,...,0.082580,-0.029094,-0.000040,-0.000503,0.142232,0.283327,0.236410,0.158867,0.188260,0.577034


In [21]:
y_pred = predict_for_dataset(x_test,kmeans_model,matrix) 

In [22]:
y_pred=np.array(y_pred)
y_test=np.array(y_test)

In [23]:
y_test.shape

(302, 22)

In [24]:
y_pred_arr=[]
for i in y_pred:
    temp=[0 for _ in range(y_test.shape[1])]
    for j in i:
        temp[int(j[1]-x_col_size)]=j[0]
    y_pred_arr.append(temp)
y_pred_arr=np.array(y_pred_arr)
y_pred_arr.shape


(302, 22)

In [25]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred_arr)

0.03010920139917702

In [26]:
ratings=pd.read_csv('ml-1m/ratings.dat', delimiter='::', names=['UserID','MovieID','Rating','TimeStamp'])
movies=pd.read_csv('ml-1m/movies.dat', delimiter='::', names=["Title","Genres"],  encoding="ISO-8859-1")

/tmp/ipykernel_23310/2496518303.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings=pd.read_csv('ml-1m/ratings.dat', delimiter='::', names=['UserID','MovieID','Rating','TimeStamp'])
/tmp/ipykernel_23310/2496518303.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies=pd.read_csv('ml-1m/movies.dat', delimiter='::', names=["Title","Genres"],  encoding="ISO-8859-1")


In [27]:
# movies
ratings.drop(['UserID','TimeStamp'], axis=1, inplace= True)

In [28]:
ratings

,MovieID,Rating
0,1193,5
1,661,3
2,914,3
3,3408,4
4,2355,5
...,...,...
1000204,1091,1
1000205,1094,5
1000206,562,5
1000207,1096,4


In [29]:
movies['Rating']=[[] for _ in range(len(movies.index))]
for i in ratings.index:
    temp=ratings.iloc[i]['Rating']
    movies.at[ratings.iloc[i]['MovieID'],'Rating'].append(temp)

In [30]:
movies['Rating']=movies['Rating'].apply(lambda x: np.array(x).mean())

/tmp/ipykernel_23310/1822539475.py:1: RuntimeWarning: Mean of empty slice.
  movies['Rating']=movies['Rating'].apply(lambda x: np.array(x).mean())
/home/netradeepakc/anaconda3/envs/rec_sys_mini/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [31]:
movies.isna().sum()

Title       0
Genres      0
Rating    177
dtype: int64

In [32]:
movies['Genres']=movies['Genres'].apply(lambda x: x.split('|'))
for i in movies.index:
    title=movies.at[i,'Title']
    decade=title[-5:-2]
    decade=str(decade)+'0s'
    movies.at[i,'Genres'].append(decade)

In [33]:
movies

,Title,Genres,Rating
1,Toy Story (1995),"[Animation, Children's, Comedy, 1990s]",4.146846
2,Jumanji (1995),"[Adventure, Children's, Fantasy, 1990s]",3.201141
3,Grumpier Old Men (1995),"[Comedy, Romance, 1990s]",3.016736
4,Waiting to Exhale (1995),"[Comedy, Drama, 1990s]",2.729412
5,Father of the Bride Part II (1995),"[Comedy, 1990s]",3.006757
...,...,...,...
3948,Meet the Parents (2000),"[Comedy, 2000s]",3.635731
3949,Requiem for a Dream (2000),"[Drama, 2000s]",4.115132
3950,Tigerland (2000),"[Drama, 2000s]",3.666667
3951,Two Family House (2000),"[Drama, 2000s]",3.900000


In [34]:
movies['Affinity']=[0 for _ in range(len(movies.index))]
for i in movies.index:
    genres=movies.at[i,'Genres']
    affinity=0
    for j in genres:
        affinity+=rating_dict[j]*movies.at[i,'Rating']
    movies.at[i,'Affinity']=affinity

/tmp/ipykernel_23310/3621030468.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7.006893086161036' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  movies.at[i,'Affinity']=affinity


In [35]:
movies.sort_values('Affinity',inplace=True,ascending=False)

In [36]:
movies

,Title,Genres,Rating,Affinity
1196,Star Wars: Episode V - The Empire Strikes Back...,"[Action, Adventure, Drama, Sci-Fi, War, 1980s]",4.292977,11.667379
2028,Saving Private Ryan (1998),"[Action, Drama, War, 1990s]",4.337354,10.783529
3266,Man Bites Dog (C'est arrivé près de chez vous)...,"[Action, Comedy, Crime, Drama, 1990s]",3.655844,10.702832
1215,Army of Darkness (1993),"[Action, Adventure, Comedy, Horror, Sci-Fi, 19...",3.804233,10.657491
110,Braveheart (1995),"[Action, Drama, War, 1990s]",4.234957,10.528949
...,...,...,...,...
3650,Anguish (Angustia) (1986),"[Horror, 1980s]",NaN,NaN
3750,Boricua's Bond (2000),"[Drama, 2000s]",NaN,NaN
3829,Mad About Mambo (2000),"[Comedy, Romance, 2000s]",NaN,NaN
3856,Autumn Heart (1999),"[Drama, 1990s]",NaN,NaN


In [37]:
top_movies=list(movies['Title'])[:5]
print("Top 5 movies are:")
for i in top_movies:
    print(i)

Top 5 movies are:
Star Wars: Episode V - The Empire Strikes Back (1980)
Saving Private Ryan (1998)
Man Bites Dog (C'est arrivé près de chez vous) (1992)
Army of Darkness (1993)
Braveheart (1995)
